In [1]:
import gensim
import json
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim
import string
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
from pymystem3 import Mystem 

morph = MorphAnalyzer()
m = Mystem()

### 1) Нормализация

In [2]:
stops = set(stopwords.words('russian')) | {'gt',}
def remove_tags(text):
    return re.sub(r'<[^>]+>', '', text)

def opt_normalize(texts, top=None):
    uniq = Counter()
    for text in texts:
        uniq.update(text)
    
    norm_uniq = {word:morph.parse(word)[0].normal_form for word, _ in uniq.most_common(top)}
    
    norm_texts = []
    for text in texts:
        
        norm_words = [norm_uniq.get(word) for word in text]
        norm_words = [word for word in norm_words if word and word not in stops]
        norm_texts.append(norm_words)
        
    return norm_texts

def tokenize(text):
    words = [word.strip(string.punctuation) for word in text.split()]
    words = [word for word in words if word]
    
    return words

Пробовал делать лемматизацию с помощью mystem в качестве эксперимента, но она занимает очень много времени.
norm_uniq = {word:m.lemmatize(word)[0] for word, _ in uniq.most_common(top)}

In [3]:
texts = open('habr_texts.txt', encoding='utf-8').read().splitlines()
texts = opt_normalize([tokenize(remove_tags(text.lower())) for text in texts], 30000)

### 2) Добавление нграммов

In [4]:
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.4) # threshold можно подбирать
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts]

### 3) Создание словаря

In [5]:
dictinary = gensim.corpora.Dictionary(ngrammed_texts)

In [6]:
dictinary.filter_extremes(no_above=0.1, no_below=20)
dictinary.compactify()

### 4) LDA

#### вариант 1

In [7]:
corpus = [dictinary.doc2bow(text) for text in ngrammed_texts]

In [8]:
lda = gensim.models.LdaMulticore(corpus, 100, id2word=dictinary, eval_every=0)

In [9]:
lda.print_topics()

[(74,
  '0.009*"дата-центр" + 0.004*"ноутбук" + 0.003*"ибп" + 0.003*"батарея" + 0.003*"температура" + 0.003*"напряжение" + 0.003*"мощность" + 0.003*"энергия" + 0.003*"аккумулятор" + 0.003*"частота"'),
 (64,
  '0.010*"текстура" + 0.003*"транзакция" + 0.003*"символ" + 0.002*"координата" + 0.002*"сканирование" + 0.002*"сетка" + 0.002*"пиксель" + 0.002*"кадр" + 0.002*"атака" + 0.002*"проектирование"'),
 (20,
  '0.003*"маска" + 0.003*"научный" + 0.002*"курс" + 0.002*"звук" + 0.002*"microsoft" + 0.002*"индекс" + 0.002*"нейросеть" + 0.002*"отчёт" + 0.002*"провайдер" + 0.002*"вм"'),
 (56,
  '0.005*"шаблон" + 0.004*"доклад" + 0.004*"робот" + 0.004*"end" + 0.003*"оператор" + 0.003*"is" + 0.002*"id" + 0.002*"as" + 0.002*"public" + 0.002*"камера"'),
 (96,
  '0.004*"microsoft" + 0.004*"контент" + 0.003*"заказчик" + 0.002*"windows" + 0.002*"•" + 0.002*"провайдер" + 0.002*"программный_обеспечение" + 0.002*"трафик" + 0.002*"github" + 0.002*"служба"'),
 (94,
  '0.007*"боль" + 0.005*"мозг" + 0.005*"тэг"

#### вариант 2 

In [10]:
lda2 = gensim.models.LdaMulticore(corpus, 100, id2word=dictinary, eval_every=0, passes = 10)

In [11]:
lda2.print_topics()

[(70,
  '0.016*"вода" + 0.015*"ручка" + 0.009*"волна" + 0.008*"температура" + 0.007*"float" + 0.007*"вод" + 0.007*"пластик" + 0.007*"прямоугольник" + 0.006*"метка" + 0.006*"голосование"'),
 (58,
  '0.022*"драйвер" + 0.010*"кэширование" + 0.008*"3d" + 0.008*"ос" + 0.008*"файловый_система" + 0.008*"фильтр" + 0.007*"звук" + 0.006*"пользовательский" + 0.005*"контекст" + 0.005*"беспроводный"'),
 (2,
  '0.013*"мастер" + 0.013*"шрифт" + 0.008*"репликация" + 0.008*"const" + 0.007*"иб" + 0.006*"функциональный" + 0.006*"блокировка" + 0.006*"public" + 0.005*"function" + 0.005*"add"'),
 (85,
  '0.054*"товар" + 0.034*"кластер" + 0.012*"def" + 0.011*"бд" + 0.010*"скидка" + 0.010*"val" + 0.009*"категория" + 0.007*"нода" + 0.007*"var" + 0.006*"value"'),
 (65,
  '0.079*"камера" + 0.011*"профиль" + 0.009*"кофе" + 0.006*"ip" + 0.005*"нейросеть" + 0.005*"кандидат" + 0.004*"словарь" + 0.004*"переводчик" + 0.004*"видеонаблюдение" + 0.003*"электронный"'),
 (52,
  '0.017*"name" + 0.011*"is" + 0.010*"id" + 0.0

#### вариант 3 

In [12]:
lda3 = gensim.models.LdaModel(corpus, 200, id2word=dictinary, eval_every=0, passes = 5, alpha='auto')

In [13]:
lda3.print_topics()

[(6,
  '0.000*"etc" + 0.000*"data" + 0.000*"backup" + 0.000*"15_декабрь" + 0.000*"id" + 0.000*"from" + 0.000*"add" + 0.000*"false" + 0.000*"биржа" + 0.000*"сигнал"'),
 (106,
  '0.441*"нейросеть" + 0.206*"обучение" + 0.103*"рассказ" + 0.054*"обучить" + 0.043*"deep" + 0.011*"светов_год" + 0.007*"несколько_десяток" + 0.006*"выдавать" + 0.005*"стратегия" + 0.004*"вход"'),
 (112,
  '0.220*"интернет_вещий" + 0.215*"raspberry_pi" + 0.128*"промышленный" + 0.084*"безопасный" + 0.081*"умный_дом" + 0.045*"переключать" + 0.040*"gpio" + 0.024*"кафедра" + 0.016*"гаджет" + 0.013*"аутентификация"'),
 (108,
  '0.167*"анализатор" + 0.133*"pvs-studio" + 0.047*"лента" + 0.040*"статический_анализ" + 0.036*"диагностика" + 0.025*"воплотить" + 0.023*"статический_анализатор" + 0.020*"разметка" + 0.020*"hdmi" + 0.019*"django"'),
 (158,
  '0.685*"задание" + 0.049*"телеграм" + 0.032*"best" + 0.028*"me" + 0.022*"15_декабрь" + 0.016*"1000" + 0.013*"call" + 0.012*"судить" + 0.006*"чат" + 0.005*"подсказка"'),
 (46,
 

#### вариант 4 

In [22]:
lda4 = gensim.models.LdaModel(corpus, 200, id2word=dictinary, eval_every=1, passes = 5, eta=2)

In [23]:
lda4.print_topics()

[(5,
  '0.000*"профессионал" + 0.000*"работодатель" + 0.000*"tdd" + 0.000*"проектирование" + 0.000*"макет" + 0.000*"обязанность" + 0.000*"заказчик" + 0.000*"срок" + 0.000*"«я" + 0.000*"ответственность"'),
 (11,
  '0.000*"функционально" + 0.000*"чистота" + 0.000*"следование" + 0.000*"сменить" + 0.000*"спрятать" + 0.000*"удовлетворять" + 0.000*"приведение" + 0.000*"стремление" + 0.000*"дорабатывать" + 0.000*"практический_применение"'),
 (187,
  '0.000*"функционально" + 0.000*"чистота" + 0.000*"следование" + 0.000*"сменить" + 0.000*"спрятать" + 0.000*"удовлетворять" + 0.000*"приведение" + 0.000*"стремление" + 0.000*"дорабатывать" + 0.000*"практический_применение"'),
 (125,
  '0.000*"design" + 0.000*"adobe" + 0.000*"дизайнер" + 0.000*"проектирование" + 0.000*"пользовательский" + 0.000*"ux" + 0.000*"слоить" + 0.000*"designer" + 0.000*"user" + 0.000*"совет"'),
 (198,
  '0.000*"ия" + 0.000*"amazon" + 0.000*"работник" + 0.000*"индия" + 0.000*"•" + 0.000*"форум" + 0.000*"труд" + 0.000*"тэг" + 0

### 5) Темы самой хорошей модели

Самой хорошей моделью с хорошо различимыми темами на мой взгляд оказался вариант 3.

lda3 = gensim.models.LdaModel(corpus, 200, id2word=dictinary, eval_every=0, passes = 5, alpha='auto')

3 хорошие темы из модели:

1) Виртульная реальность (188,'0.153*"vr" + 0.145*"виртуальный_реальность" + 0.038*"реальность" + 0.036*"дополнить_реальность" + 0.027*"еда" + 0.026*"симуляция" + 0.023*"зрение" + 0.017*"сегмент" + 0.015*"погружение" + 0.015*"ощущение"')

2) Видеоигры (115,'0.080*"игрок" + 0.033*"играть" + 0.023*"игровой" + 0.009*"дизайнер" + 0.008*"жанр" + 0.008*"аудитория" + 0.005*"valve" + 0.005*"геймплей" + 0.005*"придумать" + 0.005*"нравиться"')

3) 3d-печать (36,'0.524*"дорога" + 0.130*"ехать" + 0.111*"макет" + 0.081*"производство" + 0.036*"маршрут" + 0.029*"3d-печать" + 0.003*"особенный" + 0.002*"назначение" + 0.002*"3d-принтер" + 0.002*"распечатать"')

### 6) Добавление TF-IDF

In [24]:
tfidf_model = gensim.models.TfidfModel(corpus, id2word=dictinary)

In [25]:
corpus2 = tfidf_model[corpus]

### 7) LDA на преобразованном корпусе

In [32]:
lda5 = gensim.models.LdaModel(corpus2, 200, id2word=dictinary, eval_every=0, passes = 5, alpha='auto')

In [33]:
lda5.print_topics()

[(19,
  '0.000*"функционально" + 0.000*"чистота" + 0.000*"следование" + 0.000*"сменить" + 0.000*"спрятать" + 0.000*"удовлетворять" + 0.000*"приведение" + 0.000*"стремление" + 0.000*"дорабатывать" + 0.000*"практический_применение"'),
 (22,
  '0.000*"функционально" + 0.000*"чистота" + 0.000*"следование" + 0.000*"сменить" + 0.000*"спрятать" + 0.000*"удовлетворять" + 0.000*"приведение" + 0.000*"стремление" + 0.000*"дорабатывать" + 0.000*"практический_применение"'),
 (53,
  '0.000*"функционально" + 0.000*"чистота" + 0.000*"следование" + 0.000*"сменить" + 0.000*"спрятать" + 0.000*"удовлетворять" + 0.000*"приведение" + 0.000*"стремление" + 0.000*"дорабатывать" + 0.000*"практический_применение"'),
 (33,
  '0.000*"функционально" + 0.000*"чистота" + 0.000*"следование" + 0.000*"сменить" + 0.000*"спрятать" + 0.000*"удовлетворять" + 0.000*"приведение" + 0.000*"стремление" + 0.000*"дорабатывать" + 0.000*"практический_применение"'),
 (66,
  '0.000*"функционально" + 0.000*"чистота" + 0.000*"следование

### 8) Как изменилась модель

В новом варианте с использованием tf-idf модели видно повторение одной и той же темы несколько раз. Однако другие темы выглядят достаточно осмысленными и их можно различить и описать. Например, 
(122,'0.051*"выборка" + 0.043*"предприятие" + 0.040*"пилот" + 0.036*"актив" + 0.030*"посетитель" + 0.030*"товар" + 0.029*"рубль" + 0.025*"сделка" + 0.022*"акция" + 0.020*"потребитель"') наверняка связана с описанием бизнеса или компании. 

А эта (128,'0.052*"иконка" + 0.044*"паттерн" + 0.042*"2017" + 0.040*"css" + 0.036*"ios" + 0.033*"шаблон" + 0.033*"dropbox" + 0.030*"анимация" + 0.029*"опрос" + 0.029*"публиковать"') связана с html версткой.

In [34]:
lda3.log_perplexity(corpus[:10000]) 

-31.415426440581857

In [57]:
topics = []
for topic_id, topic in lda.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)

In [58]:
coherence_model_lda = gensim.models.CoherenceModel(model=lda, 
                                                  texts=texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [59]:
coherence_model_lda.get_coherence()

nan

In [38]:
lda5.log_perplexity(corpus2[:10000])

-180.2070180247443

In [39]:
topics = []
for topic_id, topic in lda5.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)

In [40]:
coherence_model_lda5 = gensim.models.CoherenceModel(model=lda5, 
                                                  texts=texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [41]:
coherence_model_lda5.get_coherence()

nan

Исходя из метрик перплексии более хорошей является модель варианта 3. Попытался сделать когерентность по образцу из ноутбука, но почему-то результат не выводится.

### 9) NMF

#### вариант 1 с TfidfVectorizer

In [42]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

In [43]:
stexts = [' '.join(text) for text in texts]

In [44]:
vectorizer = TfidfVectorizer(max_features=1000, min_df=10, max_df=0.3, ngram_range=(1,3))
X = vectorizer.fit_transform(stexts)

In [45]:
model = NMF(n_components=30)

In [46]:
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=30, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [47]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [48]:
feat_names = vectorizer.get_feature_names()

In [49]:
top_words = model.components_.argsort()[:,:-5:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

0 клиент  продукт  бизнес  сотрудник
1 if  int  amp  return
2 игра  игрок  игровой  играть
3 файл  папка  скрипт  настройка
4 сигнал  звук  частота  напряжение
5 lt  gt  lt div  div
6 устройство  смартфон  мобильный  умный
7 the  to  of  and
8 public  new  void  string
9 уязвимость  атака  безопасность  защита
10 сайт  страница  реклама  браузер
11 react  javascript  js  компонент
12 космический  аппарат  земля  станция
13 доклад  конференция  участник  тема
14 рубль  рынок  цена  миллион
15 процессор  память  intel  диск
16 печать  3d  материал  модель
17 учёный  мозг  исследование  исследователь
18 объект  значение  метод  элемент
19 язык  программирование  программа  программист
20 сеть  связь  трафик  интернет
21 сервер  дата центр  дата  сервис
22 бот  сообщение  канал  робот
23 end  запрос  from  as
24 google  android  мобильный  ios
25 вселенная  энергия  свет  теория
26 камера  видео  смартфон  изображение
27 виртуальный  машина  виртуальный машина  windows
28 книга  какой то  

In [50]:
model.reconstruction_err_

54.961388645549114

#### вариант 2 с CountVetcorizer

In [51]:
vectorizer = CountVectorizer(max_features=1000, min_df=10, max_df=0.3, ngram_range=(1,3))
X = vectorizer.fit_transform(stexts)

In [52]:
model = NMF(n_components=30)

In [53]:
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=30, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [54]:
feat_names = vectorizer.get_feature_names()

In [55]:
top_words = model.components_.argsort()[:,:-5:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

0 up  2016  11  23
1 lt  gt  gt lt  function
2 sec  ms  точка  13
3 какой то  продукт  рынок  что то
4 if  bool  else  return
5 значение  переменный  поль  канал
6 the  of  to  is
7 игра  игрок  игровой  играть
8 объект  класс  метод  элемент
9 end  begin  if  then
10 адрес  mac  сеть  пакет
11 int  for  lt  amp
12 сетевой  драйвер  пакет  очередь
13 файл  папка  модуль  параметр
14 public  new  void  string
15 lt  return  val  if
16 of  and  amp  for
17 user  do  end  test
18 as  set  on  from
19 result  return  function  let
20 main  import  значение  var
21 сайт  страница  карта  строка
22 кнопка  char  нажать  иначе
23 сервер  запрос  клиент  скидка
24 модель  сеть  точка  обучение
25 type  00  data  name
26 центр  дата  дата центр  услуга
27 язык  программа  память  программирование
28 устройство  управление  мобильный  камера
29 резервный  копия  объём  канал


In [56]:
model.reconstruction_err_

2345.54934239407

#### вариант 3 с TfidfVectorizer и другими параметрами

In [61]:
vectorizer = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.2, ngram_range=(1,3))
X = vectorizer.fit_transform(stexts)

In [62]:
model = NMF(n_components=50)

In [63]:
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=50, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [64]:
feat_names = vectorizer.get_feature_names()

In [65]:
top_words = model.components_.argsort()[:,:-5:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

0 сотрудник  программист  деньга  бизнес
1 int  lt  for int  char
2 игра  игровой  играть  движок
3 плата  напряжение  температура  ток
4 смартфон  телефон  экран  дисплей
5 lt  gt  div  lt div
6 облачный  облако  услуга  инфраструктура
7 public  void  string  private
8 the  to  of  and
9 космический  спутник  орбита  ракета
10 процессор  intel  память  ядро
11 учёный  клетка  исследование  пациент
12 книга  читать  часы  профессиональный
13 доклад  конференция  участник  мероприятие
14 дата центр  центр  дата  оборудование
15 печать  3d  принтер  материал
16 обучение  ия  нейросеть  алгоритм
17 виртуальный  машина  виртуальный машина  вм
18 тест  тестирование  тестировать  тестовый
19 android  мобильный  ios  мобильный приложение
20 сигнал  частота  канал  станция
21 if  return  amp  else
22 бот  сообщение  канал  api
23 вселенная  галактика  энергия  звезда
24 windows  microsoft  linux  azure
25 запрос  таблица  база дать  from
26 реальность  виртуальный реальность  vr  виртуальный
2

### 10) Лучшие модели NMF

На мой взгляд более хорошей моделью оказался 3 вариант. Причиной этому скорее всего послужило увеличение значения параметра n_components=50, что привело к появлению большего количества осмысленных тем. 

Хороших тем получилось очень много. Например:
* 4 смартфон  телефон  экран  дисплей
* 15 печать  3d  принтер  материал
* 38 диск  запись  ssd  жёсткий
* 39 ключ  сертификат  пароль  шифрование
* 48 адрес  ip  домен  ip адрес

и др.

Я думаю, что темы NMD выглядят связаннее и понятнее, чем в LDA. 